In [2]:
# Carrega os pacotes usuais.
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math as math
import scipy.signal as ss
from scipy import optimize
from numba import jit

# Determina que o pyplot apresente a imagem no bloco executado.
%matplotlib inline
#
%config IPCompleter.greedy=True
# Determina uma quantidade máxima de linhas na exibição de um pandas DataFrame
pd.options.display.max_rows = 10

# Carrega o módulo do algoritmo de segmentação.
import segmentation_algorithm as seg

In [2]:
# Extrai o titulo dos arquivos de todos os três grupos de pacientes, a fim de preservar
# a ordem alfabética em que estão os arquivos.
# Guarda os titulos em um arquivo, na sequência lida, para recarregá-los posteriormente 
# em qualquer computador e na mesma ordem.

all_file_titles = os.listdir("dados/complete")
hypertensive_file_titles = []
normotensive_file_titles = []
proband_file_titles = []

for i in range(len(all_file_titles)):
    if "htm_" in all_file_titles[i]:
        hypertensive_file_titles.append(all_file_titles[i])
    elif "ntm_" in all_file_titles[i] or "ntb" in all_file_titles[i]:
        normotensive_file_titles.append(all_file_titles[i])
    elif "pb_" in all_file_titles[i]:
        proband_file_titles.append(all_file_titles[i])
        
ht_file_titles = open("file_titles_ht.txt", "w")
nt_file_titles = open("file_titles_nt.txt", "w")
pb_file_titles = open("file_titles_pb.txt", "w")

for i in range(len(hypertensive_file_titles)):
    hypertensive_file_titles[i] = 'dados/complete/'+hypertensive_file_titles[i][:-4]
    ht_file_titles.write(hypertensive_file_titles[i]+"\n")
for i in range(len(normotensive_file_titles)):
    normotensive_file_titles[i] = 'dados/complete/'+normotensive_file_titles[i][:-4]
    nt_file_titles.write(normotensive_file_titles[i]+"\n")
for i in range(len(proband_file_titles)):
    proband_file_titles[i] = 'dados/complete/'+proband_file_titles[i][:-4]
    pb_file_titles.write(proband_file_titles[i]+"\n")
    
ht_file_titles.close()
nt_file_titles.close()
pb_file_titles.close()

In [3]:
# Carrega os titulos dos arquivos salvos anteriormente

ht_file_titles = open("file_titles_ht.txt", "r")
nt_file_titles = open("file_titles_nt.txt", "r")
pb_file_titles = open("file_titles_pb.txt", "r")
hypertensive_file_titles = [x.strip() for x in ht_file_titles.readlines()]
normotensive_file_titles = [x.strip() for x in nt_file_titles.readlines()]
proband_file_titles      = [x.strip() for x in pb_file_titles.readlines()]
ht_file_titles.close(); nt_file_titles.close(); pb_file_titles.close()

In [4]:
# Carrega e guarda os dados de todos os pacientes de cada grupo em uma lista de dataframes

hypertensive_data = []
normotensive_data = []
proband_data = []
titulos = ['Tempo', 'Serie 0', 'Serie 1', 'Serie 2', 'Serie 3', 'SleepStage']

for i in range(len(hypertensive_file_titles)):
    brute_data = pd.read_table(hypertensive_file_titles[i]+".dat", sep='\t', names = range(20))
    hypertensive_data.append(brute_data[[1, 2, 3, 4, 5, 11]])
    hypertensive_data[i].columns = titulos
for i in range(len(normotensive_file_titles)):
    brute_data = pd.read_table(normotensive_file_titles[i]+".dat", sep='\t', names = range(20))
    normotensive_data.append(brute_data[[1, 2, 3, 4, 5, 11]])
    normotensive_data[i].columns = titulos
for i in range(len(proband_file_titles)):
    brute_data = pd.read_table(proband_file_titles[i]+".dat", sep='\t', names = range(20))
    proband_data.append(brute_data[[1, 2, 3, 4, 5, 11]])
    proband_data[i].columns = titulos

C:\Users\Patrick\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# calcula os saltos das séries.

series = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
grupos = [hypertensive_data, normotensive_data, proband_data]
for dados in grupos:
    for k in series:
        for j in range(len(dados)):
            saltos = np.zeros(len(dados[j]))
            for i in range(len(dados[j]) - 1):
                saltos[i] = dados[j][k].loc[i+1] - dados[j][k].loc[i]

            saltos[len(dados[j]) - 1] = np.nan
            dados[j][k+" Saltos"] = saltos
            
# Salva os dados com os saltos em outra pasta

for i in range(len(hypertensive_data)):
    hypertensive_data[i].to_csv("dados/dados_brutos_com_saltos/"+hypertensive_file_titles[i][15:]+".csv")
for i in range(len(normotensive_data)):
    normotensive_data[i].to_csv("dados/dados_brutos_com_saltos/"+normotensive_file_titles[i][15:]+".csv")
for i in range(len(proband_data)):
    proband_data[i].to_csv("dados/dados_brutos_com_saltos/"+proband_file_titles[i][15:]+".csv")

In [ ]:
# Aplica a segmentação nos dados (em forma matricial).
titulos = ['Tempo', 'Serie 0', 'Serie 1', 'Serie 2', 'Serie 3']

# Defini os grupos cujas séries serão segmentadas.
group = "proband"
data = proband_data
group_file_titles = proband_file_titles

result_dataframes = []
for i in range(len(data)):
    temp = [pd.DataFrame()]*4
    print("Subject", i)
    for j in range(4):
        print("\n Segmenting")
        temp[j] = seg.segment(data[i][titulos[j+1]].values)
    
    result_dataframes.append(temp.copy())
    print("Finished segmentation of the subject", i,"data.")
    
    
# Classificação de todas as segmentações por fases do sono.
# A classificação é feita considerando a fase de maior incidência no segmento.
for i in range(len(data)):
    for j in range(4):
        for k in list(result_dataframes[i][j].index):
            start = int(result_dataframes[i][j]["start"][k])
            finish = int(result_dataframes[i][j]["finish"][k])
            result_dataframes[i][j].loc[k:k, "SleepStage"] = int(data[i]["SleepStage"][start:finish+1].value_counts().idxmax())    
    

# guarda os dados em .csv
for i in range(len(data)):
    for j in range(4):
        result_dataframes[i][j].to_csv("dados/segmentation_dataframes_"+group+"/"+group_file_titles[i][15:][:-4]+"_"+titulos[j+1]+".csv")

In [3]:
# carrega os dataframes dos dados brutos e dos resultados da segmentação

text_files = [open("file_titles_ht.txt", "r"), open("file_titles_nt.txt", "r"), open("file_titles_pb.txt", "r")]
file_titles = [[x.strip() for x in text_files[0].readlines()],
               [x.strip() for x in text_files[1].readlines()],
               [x.strip() for x in text_files[2].readlines()]]
text_files[0].close(); text_files[1].close(); text_files[2].close()


number_of_subjects = [len(file_titles[0]), len(file_titles[1]), len(file_titles[2])]
seg_res = [[], [], []]
data = [[], [], []]
for j in [0, 1, 2]:
    for i in range(number_of_subjects[j]):
        data[j].append(pd.read_csv("dados/dados_brutos_com_saltos/"+file_titles[j][i][15:]+".csv", index_col = 0))

folder = ["dados/segmentation_dataframes_hypertensive/",
          "dados/segmentation_dataframes_normotensive/",
          "dados/segmentation_dataframes_proband/"]
series = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
for j in [0, 1, 2]:    
    for i in range(number_of_subjects[j]):
        temp = [pd.DataFrame()]*4

        for k in range(4):
            temp[k] = pd.read_csv(folder[j]+file_titles[j][i][15:][:-4]+"_"+series[k]+".csv").set_index("index")
            length = len(temp[k].index)

        seg_res[j].append(temp.copy())

A filtragem das séries consiste em percorrer cada segmento, calculando, primeiramente, os saltos nos extremos 0.85 e 0.15% da série (o salto no ponto i corresponde à diferença entre i e i-1), trocando os valores correspondentes para a mediana do segmento, e recalculando a média, por fim.

Os saltos são calculados novamente, e o algoritmo de filtragem executado mais uma vez, buscando saltos nos extremos 0.98 e 0.02%.

*****

Posso me questionar se há a necessidade de aplicar novamente o algoritmo de segmentação, pela possibilidade de que o resultado se altere, particularmente nas séries em que a incidência de outliers era muito grande. Posso pensar que, se a filtragem que fiz foi eficiente, o algoritmo de segmentação é bastante útil para alguma técnica de filtragem em que janelas dentro da série devem ser escolhidas, e não é conveniente filtrar a série inteira de uma vez só.

Quais outros mecanismos de filtragem de séries temporais existem?

Todos baseados em uma janela móvel que substitui valores pela mediana da janela, análogo ao modo que apliquei. O único que compensa ser testado é o filtro de Hampel.

In [115]:
# Filtra a série
#    Já Filtrada

# Definição da função de filtragem.

def filtro(qmax):
    title = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
    title_jp = ["Serie 0 Saltos", "Serie 1 Saltos", "Serie 2 Saltos", "Serie 3 Saltos"]
    for i in range(3):
        for j in range(len(seg_res[i])):
            for k in range(4):
                serie = data[i][j][title[k]]
                for l in range(len(seg_res[i][j][k])):
                    start = int(seg_res[i][j][k].loc[l+1]["start"])
                    finish = int(seg_res[i][j][k].loc[l+1]["finish"])
                    sslice = data[i][j][title[k]][start: finish+1]
                    sslice_jumps = data[i][j][title_jp[k]][start: finish+1]

                    maxpercentile = sslice_jumps.quantile(qmax)
                    minpercentile = sslice_jumps.quantile(1-qmax)
                    sslice_jumps_id = sslice_jumps[ (sslice_jumps < minpercentile) | (sslice_jumps > maxpercentile)].index
                    sslice[sslice_jumps_id] = sslice.median()
                    sslice[sslice_jumps_id] = sslice.median()

                    seg_res[i][j][k].loc[l+1]["mean"] = sslice.mean()
                    

# Definição da função de cálculo dos saltos
def calc_saltos():
    series = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
    for dados in data:
        for k in series:
            for j in range(len(dados)):
                saltos = np.zeros(len(dados[j]))
                for i in range(len(dados[j]) - 1):
                    saltos[i] = dados[j][k].loc[i+1] - dados[j][k].loc[i]

                saltos[len(dados[j]) - 1] = np.nan
                dados[j][k+" Saltos"] = saltos

# Primeira passagem
filtro(0.85)
calc_saltos()
# Segunda passagem
filtro(0.98)
calc_saltos()

# Salva os dados.

for i in range(len(data[0])):
    data[0][i].to_csv("dados/dados_brutos_com_saltos/"+hypertensive_file_titles[i][15:]+".csv")
for i in range(len(data[1])):
    data[1][i].to_csv("dados/dados_brutos_com_saltos/"+normotensive_file_titles[i][15:]+".csv")
for i in range(len(data[2])):
    data[2][i].to_csv("dados/dados_brutos_com_saltos/"+proband_file_titles[i][15:]+".csv")

In [ ]:
# Modelo de aplicação do filtro de Hampel

# Definição da função de Hampel
# Créditos: Eduardo Osorio,
# https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-hampel-function-faster
def hampel(vals_orig, k=7, t0=3):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''

    #Make copy so original not edited
    vals = vals_orig.copy()

    #Hampel Filter
    L = 1.4826
    rolling_median = vals.rolling(window=k, center=True).median()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_MAD = vals.rolling(window=k, center=True).apply(MAD)
    threshold = t0 * L * rolling_MAD
    difference = np.abs(vals - rolling_median)

    '''
    Perhaps a condition should be added here in the case that the threshold value
    is 0.0; maybe do not mark as outlier. MAD may be 0.0 without the original values
    being equal. See differences between MAD vs SDV.
    '''

    outlier_idx = difference > threshold
    vals[outlier_idx] = np.nan
    return(vals)

serie_t = normotensive_data[6]["Serie 2"]
tempo = normotensive_data[6]["Tempo"]
fig, axs = plt.subplots(nrows = 3, ncols = 1)
fig.set_size_inches(40, 45)

for i in range(10):
    serie_t = hampel(serie_t)

axs[0].plot(normotensive_data[6]["Tempo"], normotensive_data[6]["Serie 2"])
axs[0].set_title("Série original")

axs[1].plot(data[1][6]["Tempo"], data[1][6]["Serie 2"])
axs[1].set_title("Filtrada após segmentação")

axs[2].plot(tempo, serie_t)
axs[2].set_title("Filtrada pela função de Hampel")

In [9]:
# Conta a proporção de fases em cada segmento
#   Já computado.

s_titles = ["SBP", "DBP", "Procure1", "Procure2"]
for i in range(3):
    for j in range(len(seg_res[i])):
        for k in range(4):
            seg_size = len(seg_res[i][j][k])
            stage_count = pd.DataFrame({"0_%": np.zeros(seg_size), "1_%": np.zeros(seg_size), 
                                        "2_%": np.zeros(seg_size), "3_%": np.zeros(seg_size), "4_%": np.zeros(seg_size), 
                                        "REM_%": np.zeros(seg_size), "66_%": np.zeros(seg_size)})
            stage_count.index += 1
            for l in range(seg_size):
                seg_start = int(seg_res[i][j][k].loc[l+1]["start"])
                seg_finish = int(seg_res[i][j][k].loc[l+1]["finish"])
                seg_slice = data[i][j][seg_start:seg_finish+1]
                seg_slice_count = seg_slice["SleepStage"].value_counts()
                if 0.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["0_%"] = np.round(seg_slice_count[0.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 1.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["1_%"] = np.round(seg_slice_count[1.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 2.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["2_%"] = np.round(seg_slice_count[2.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 3.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["3_%"] = np.round(seg_slice_count[3.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 4.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["4_%"] = np.round(seg_slice_count[4.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 5.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["REM_%"] = np.round(seg_slice_count[5.0]/(seg_finish - seg_start + 1), decimals = 3)
                if 66.0 in seg_slice_count.index:
                    stage_count.loc[l+1]["66_%"] = np.round(seg_slice_count[66.0]/(seg_finish - seg_start + 1), decimals = 3)
                    
            seg_res[i][j][k] = seg_res[i][j][k].join(stage_count)
            
            


In [6]:
# salva os dataframes dos segmentos

grupos = ["hypertensive", "normotensive", "proband"]
serie  = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
pasta  = ["segmentation_dataframes_hypertensive", "segmentation_dataframes_normotensive",
          "segmentation_dataframes_proband"]
for i in range(3):
    for j in range(len(file_titles[i])):
        title = file_titles[i][j][15:][:-4]
        for k in range(4):
            seg_res[i][j][k].to_csv("dados/"+pasta[i]+"/"+title+"_"+serie[k]+".csv")

In [25]:
# Plota as séries

# Definição da função para gerar as cores de fundo
def axvlines(xs, c, max_val, ax=None, **plot_kwargs):
    """
    Draw vertical lines on plot
    :param xs: A scalar, list, or 1D array of horizontal offsets
    :param ax: The axis (or none to use gca)
    :param plot_kwargs: Keyword arguments to be passed to plot
    :return: The plot object corresponding to the lines.
    """
    if ax is None:
        ax = plt.gca()
    xs = np.array((xs, ) if np.isscalar(xs) else xs, copy=False)
    lims = ax.get_ylim()
    x_points = np.repeat(xs[:, None], repeats=3, axis=1).flatten()
    y_points = np.repeat(np.array(lims + (np.nan, ))[None, :], repeats=len(xs), axis=0).flatten()
    plot = ax.plot(x_points, max_val*y_points, scaley = False, **plot_kwargs, color = c)
    return plot

colors  = ["gold", "lavender", "lightsteelblue", "thistle", "orchid", "lightcoral"]
t_serie = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
t_serie_nome = ["SBP", "DBP", "BBI-BP (ms)", "BBI-KG (ms)"]
t_grupo = ["Hypertensive", "Normotensive", "Proband"]

for grupo in [0, 1, 2]:
    for individuo in range(len(data[grupo])):
        fig, axs = plt.subplots(nrows = 4, ncols = 1)
        fig.set_size_inches(30, 25)
        fig.subplots_adjust(hspace = 0.3, wspace = 0.5)
        
        for serie in [0, 1, 2, 3]:
            axis = axs[serie]
            result = seg_res[grupo][individuo][serie]
            series = data[grupo][individuo][t_serie[serie]]
            time = data[grupo][individuo]["Tempo"]
            means = np.zeros(0)
            max_val = series.max()
            min_val = series.min()

            for i in result.index:
                size = int(result.loc[i,'finish'] - result.loc[i, 'start'] + 1)
                vec = np.full([size], result.loc[i, 'mean'])
                means = np.concatenate((means,vec))
            
            for i in range(6):
                axvlines(data[grupo][individuo][data[grupo][individuo]["SleepStage"] == float(i)]["Tempo"].values, colors[i], max_val, ax = axis)

            axis.plot(time, series, color = 'dimgray')
            axis.plot(time, means, 'r-', linewidth = 2, drawstyle='steps-post', label='step-post')
            axis.set_xticks(range(0, int(time[len(time)-1]), 5000))
            axis.set_ylim(int(min_val) - 20, int(max_val))
            axis.set_xlim(time[0], time[len(time)-1])
            axis.set_xlabel('tempo (ms)', fontsize = 20)
            axis.set_ylabel('intervalo entre batimentos (s)', fontsize = 20)
            axis.set_title(t_serie_nome[serie], fontsize = 25)
        
        fig.savefig("Plots/Series - Final, Com Cores de Fundo/"+t_grupo[grupo]+"/Subject"+str(individuo)+".png")
        plt.close()

#### Análise de fato.

In [10]:
# Calcula as medianas dos segmentos

series = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
for grupo in range(3):
    for indiv in range(len(seg_res[grupo])):
        for serie in range(len(seg_res[grupo][indiv])):
            resultado = seg_res[grupo][indiv][serie]
            mediana = np.zeros(len(resultado))
            
            for index in resultado.index:
                start = int(resultado["start"][index])
                finish = int(resultado["finish"][index])
                
                mediana[index - 1] = data[grupo][indiv][series[serie]][start: finish+1].median()
            
            resultado["Median"] = mediana

In [11]:
# Calcula o tempo de início, fim e duração de cada segmento.

for grupo in range(3):
    for indiv in range(len(seg_res[grupo])):
        for serie in range(len(seg_res[grupo][indiv])):
            t_i = np.zeros(len(seg_res[grupo][indiv][serie]))
            t_f = np.zeros(len(seg_res[grupo][indiv][serie]))
            dt  = np.zeros(len(seg_res[grupo][indiv][serie]))
            L   = np.zeros(len(seg_res[grupo][indiv][serie]))
            
            for index in range(1, len(seg_res[grupo][indiv][serie])+1):
                start  = int(seg_res[grupo][indiv][serie]["start"][index])
                finish = int(seg_res[grupo][indiv][serie]["finish"][index])
                t_i[index-1] = data[grupo][indiv]["Tempo"][start]
                t_f[index-1] = data[grupo][indiv]["Tempo"][finish]
                dt[index-1]  = data[grupo][indiv]["Tempo"][finish] - data[grupo][indiv]["Tempo"][start]
                L[index-1]   = finish - start
            
            seg_res[grupo][indiv][serie]["T_i"] = t_i
            seg_res[grupo][indiv][serie]["T_f"] = t_f
            seg_res[grupo][indiv][serie]["Dt"]  = dt
            seg_res[grupo][indiv][serie]["L"]   = L    

In [4]:
# Calcula o tempo de início, fim e duração de cada segmento.

for grupo in range(3):
    for indiv in range(len(seg_res[grupo])):
        for serie in range(len(seg_res[grupo][indiv])):
            dt  = np.zeros(len(seg_res[grupo][indiv][serie]))
            
            for index in range(1, len(seg_res[grupo][indiv][serie])+1):
                start  = int(seg_res[grupo][indiv][serie]["start"][index])
                finish = int(seg_res[grupo][indiv][serie]["finish"][index])
                dt[index-1]  = finish - start
            
            seg_res[grupo][indiv][serie]["L"]  = dt
                

In [ ]:
# deleta os segmentos com fase indeterminada ou com frequência máxima menor que 90%

for i in range(3):
    for j in range(len(seg_res[i])):
        for k in range(4):
            seg_res[i][j][k] = seg_res[i][j][k].drop(seg_res[i][j][k][seg_res[i][j][k]["SleepStage"] == 66.0].index)
            for l in seg_res[i][j][k].index:
                if max(seg_res[i][j][k].loc[l, "0_%":"66_%"]) < 0.9:
                    seg_res[i][j][k] = seg_res[i][j][k].drop(l)

In [ ]:
# deleta os segmentos com fase indeterminada ou com frequência máxima menor que 90%

for i in range(3):
    for j in range(len(seg_res[i])):
        for k in range(4):
            seg_res[i][j][k] = seg_res[i][j][k].drop(seg_res[i][j][k][seg_res[i][j][k]["SleepStage"] == 66.0].index)
            for l in seg_res[i][j][k].index:
                if max(seg_res[i][j][k].loc[l, "0_%":"66_%"]) < 0.85:
                    seg_res[i][j][k] = seg_res[i][j][k].drop(l)

# aglutinação das estatísticas dos segmentos de todos os sujeitos para a série SBP

medias     = [[], [], []]
medianas   = [[], [], []]
variancias = [[], [], []]
tamanhos   = [[], [], []]
t_inicio   = [[], [], []]
t_final    = [[], [], []]
delta_t    = [[], [], []]

for grupo in [0, 1, 2]:
    medias_series_temp     = [[], [], [], []]
    medianas_series_temp   = [[], [], [], []]
    variancias_series_temp = [[], [], [], []]
    tamanhos_series_temp   = [[], [], [], []]
    t_inicio_series_temp   = [[], [], [], []]
    t_final_series_temp    = [[], [], [], []]
    delta_t_series_temp    = [[], [], [], []]
    
    for serie in [0, 1, 2, 3]:
        # Isso pode ser simplificado se eu reenumerar as fases
        medias_temp     = [[], [], [], [], [], []]
        medianas_temp   = [[], [], [], [], [], []]
        variancias_temp = [[], [], [], [], [], []]
        tamanhos_temp   = [[], [], [], [], [], []]
        t_inicio_temp   = [[], [], [], [], [], []]
        t_final_temp    = [[], [], [], [], [], []]
        delta_t_temp    = [[], [], [], [], [], []]

        for i in range(len(data[grupo])): 
            df = seg_res[grupo][i][serie]  
            for j in [0, 1, 2, 3, 4, 5]:
                for index in df.where(df["SleepStage"] == j).dropna().index:
                    medias_temp[j].append(df["mean"][index])
                    medianas_temp[j].append(df["Median"][index])
                    variancias_temp[j].append(df["variance"][index])
                    tamanhos_temp[j].append(df["finish"][index] - df["start"][index])
                    t_inicio_temp[j].append(df["T_i"][index])
                    t_final_temp[j].append(df["T_f"][index])
                    delta_t_temp[j].append(df["Dt"][index])

        # Coloca os valores em escala logaritmica.
        for i in range(6):
            variancias_temp[i] = np.log(np.asarray(variancias_temp[i]))
            tamanhos_temp[i]   = np.log(np.asarray(tamanhos_temp[i]))
            medias_temp[i]     = np.log(np.asarray(medias_temp[i]))
            medianas_temp[i]   = np.log(np.asarray(medianas_temp[i]))
            # transforma as listas restantes em arrays
            t_inicio_temp[i]   = np.asarray(t_inicio_temp[i])
            t_final_temp[i]    = np.asarray(t_final_temp[i])
            delta_t_temp[i]    = np.asarray(delta_t_temp[i])
        
        medias_series_temp[serie]     = medias_temp
        medianas_series_temp[serie]   = medianas_temp
        variancias_series_temp[serie] = variancias_temp
        tamanhos_series_temp[serie]   = tamanhos_temp
        t_inicio_series_temp[serie]   = t_inicio_temp
        t_final_series_temp[serie]    = t_final_temp
        delta_t_series_temp[serie]    = delta_t_temp
    
    medias[grupo]       = medias_series_temp
    medianas[grupo]     = medianas_series_temp
    variancias[grupo]   = variancias_series_temp
    tamanhos[grupo]     = tamanhos_series_temp
    t_inicio[grupo]     = t_inicio_series_temp
    t_final[grupo]      = t_final_series_temp
    delta_t[grupo]      = delta_t_series_temp

### Moving Averages

Técnica em que se analisa a tendência de uma séria a partir da variação da média ao longo do tempo. Consiste em mover uma janela de tamanho $n$ ao longo da série, calculando a média dos valores dentro da janela, construindo uma segunda série, a variação da média.
***

A análise das séries de batimentos cardíacos, com a finalidade de identificar automaticamente as fases do sono, a partir do algoritmo de segmentação, tem sido feita principalmente considerando que os segmentos (regiões de estacionariedade) correspondem de algum modo às fases, e essa correspondência pode ser exposta identificando diferenças das estatísticas entre os segmentos classificados e aglomerados segundo suas fases (a classificação é feita por frequência absoluta).

Até então, não foi verificada uma diferença significativa entre os segmentos que permita a identificação das fases. Posso seguir buscando outras maneiras de reduzir cada segmento a um número e comparar as diferenças entre as fases. Esses valores seriam utilizados em algum algoritmo de classificação. Ou posso buscar uma nova maneira de utilizar o algoritmo de segmentação para resolver o problema. Um meio é o de verificar diferenças de tendências entre as fases.

In [ ]:
# Calculo e plot das tendências - moving averages


# Definição da função para gerar as cores de fundo
def axvlines(xs, c, max_val, ax=None, **plot_kwargs):
    """
    Draw vertical lines on plot
    :param xs: A scalar, list, or 1D array of horizontal offsets
    :param ax: The axis (or none to use gca)
    :param plot_kwargs: Keyword arguments to be passed to plot
    :return: The plot object corresponding to the lines.
    """
    if ax is None:
        ax = plt.gca()
    xs = np.array((xs, ) if np.isscalar(xs) else xs, copy=False)
    lims = ax.get_ylim()
    x_points = np.repeat(xs[:, None], repeats=3, axis=1).flatten()
    y_points = np.repeat(np.array(lims + (np.nan, ))[None, :], repeats=len(xs), axis=0).flatten()
    plot = ax.plot(x_points, max_val*y_points, scaley = False, **plot_kwargs, color = c)
    return plot


colors  = ["gold", "lavender", "lightsteelblue", "thistle", "orchid", "lightcoral"]
t_series = ["Serie 0", "Serie 1", "Serie 2", "Serie 3"]
t_grupo = ["Hypertensive", "Normotensive", "Proband"]



for grupo in range(3):
    for indiv in range(len(data[grupo])):
        # Figura para plot das séries
        fig, axs = plt.subplots(nrows = 4, ncols = 1)
        fig.set_size_inches(40, 60)
        for serie_id in range(4):
            df = data[grupo][indiv]
            serie = df[t_series[serie_id]]
            tempo = df["Tempo"]
            mov_avg = serie.rolling(window=int(len(serie)/100)).mean()
            mov_avg.fillna(mov_avg[mov_avg.index[1]])


            max_val = mov_avg.max()
            min_val = mov_avg.min()
            for i in [0, 1, 2, 3, 4, 5]:
                axvlines(df[df["SleepStage"] == float(i)]["Tempo"].values, colors[i], max_val, ax = axs[serie_id])

            axs[serie_id].plot(tempo, mov_avg, color = 'black')

            axs[serie_id].set_xticks(range(0, int(tempo[len(tempo)-1]), 5000))
            axs[serie_id].set_ylim(int(min_val) - 10, int(max_val))
            axs[serie_id].set_xlim(tempo[0], tempo[len(tempo)-1])
            axs[serie_id].set_xlabel("Time")
            axs[serie_id].set_xlabel("Average")
            axs[serie_id].set_title(t_series[serie_id]+" - Moving Averages")
            
            
        fig.savefig("Plots/Moving Averages/"+t_grupo[grupo]+"/Individuo "+str(indiv)+".png")     


### Curva ROC 3 Fases - Eficiência de classificação via verificação direta do tamanho do segmento.

In [ ]:
media = []
tamanho = []
fase = []
todos = [[], [], []]
series = 3

# 1: pegar os valores pertinentes de todos os individuos, separados por grupos.
for j in range(3):
    for i in range(len(data[j])):
        media = media + list(seg_res[j][i][series]["mean"].values)
        tamanho = tamanho + list( seg_res[j][i][series]["finish"] - seg_res[j][i][series]["start"] )
        fase = fase + list(seg_res[j][i][series]["SleepStage"])
    todos[j] = pd.DataFrame(columns = ["media", "tamanho", "fase ref", "prev"])
    todos[j]["media"] = media; todos[j]["tamanho"] = np.log(np.asarray(tamanho)); todos[j]["fase ref"] = fase
    
# 2: refaz a classificação dos segmentos por 3 fases.    
for i in range(3):
    for j in range(len(todos[i])):
        if (todos[i]["fase ref"][j] == 1.0):
            todos[i].loc[j, "fase ref"] = 0.0
        elif (todos[i]["fase ref"][j] == 2.0 or todos[i]["fase ref"][j] == 3.0):
            todos[i].loc[j, "fase ref"] = 1.0
        elif (todos[i]["fase ref"][j] == 5.0):
            todos[i].loc[j, "fase ref"] = 2.0

# 3: separo as tabelas obtidas anteriormente pelas fases.            
separacao_fases_ht = [[], [], []]
separacao_fases_nt = [[], [], []]
separacao_fases_pb = [[], [], []]
for i in range(3):
    separacao_fases_ht[i] = todos[0][(todos[0]["fase ref"] == i)]
    separacao_fases_nt[i] = todos[1][(todos[1]["fase ref"] == i)]
    separacao_fases_pb[i] = todos[2][(todos[2]["fase ref"] == i)]

In [ ]:
# Médias dos tamanhos para cada fase

# serie 0
#ht_mean = np.array([3.85, 3.90, 4.14])
#nt_mean = np.array([3.83, 3.79, 4.18])
#pb_mean = np.array([3.73, 3.81, 3.95])

# serie 1
#ht_mean = np.array([3.91, 4.04, 4.32])
#nt_mean = np.array([3.93, 3.95, 4.32])
#pb_mean = np.array([3.82, 3.81, 4.05])

# serie 2
#ht_mean = np.array([3.77, 3.87, 4.29])
#nt_mean = np.array([3.84, 4.02, 4.72])
#pb_mean = np.array([3.89, 4.06, 4.45])

# serie 3
ht_mean = np.array([3.72, 3.87, 4.22])
nt_mean = np.array([3.84, 3.99, 4.64])
pb_mean = np.array([3.89, 4.06, 4.45])

In [ ]:
# 1: faço uma previsão por comparação das diferenças absolutas do tamanho com o tamanho médio.
@jit
def prev(ht_mean, nt_mean, pb_mean, separacao_fases_ht, separacao_fases_nt, separacao_fases_pb):
    for j in range(3):
        for i in separacao_fases_ht[j].index:
            separacao_fases_ht[j].loc[i,"prev"] = np.argmin(np.abs(ht_mean - separacao_fases_ht[j]["tamanho"].loc[i]))
        for i in separacao_fases_nt[j].index:
            separacao_fases_nt[j].loc[i,"prev"] = np.argmin(np.abs(nt_mean - separacao_fases_nt[j]["tamanho"].loc[i]))
        for i in separacao_fases_pb[j].index:
            separacao_fases_pb[j].loc[i,"prev"] = np.argmin(np.abs(pb_mean - separacao_fases_pb[j]["tamanho"].loc[i]))

In [ ]:
prev(ht_mean, nt_mean, pb_mean, separacao_fases_ht, separacao_fases_nt, separacao_fases_pb)

In [ ]:
# 2: calculos os pontos para a ROC, avaliando a eficiência da previsão para cada fase, para cada grupo        
points_ht = [[], []]
points_ht[0] = [1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]))/( len(separacao_fases_ht[1]) + len(separacao_fases_ht[2]) ),
                1 - (len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]))/( len(separacao_fases_ht[0]) + len(separacao_fases_ht[2]) ),
                1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]))/( len(separacao_fases_ht[0]) + len(separacao_fases_ht[1]) )]
points_ht[1] = [len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0])/len(separacao_fases_ht[0]),
                len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1])/len(separacao_fases_ht[1]),
                len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2])/len(separacao_fases_ht[2])]

points_nt = [[], []]
points_nt[0] = [1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]))/( len(separacao_fases_nt[1]) + len(separacao_fases_nt[2]) ),
                1 - (len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]))/( len(separacao_fases_nt[0]) + len(separacao_fases_nt[2]) ),
                1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]))/( len(separacao_fases_nt[0]) + len(separacao_fases_nt[1]) )]
points_nt[1] = [len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0])/len(separacao_fases_nt[0]),
                len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1])/len(separacao_fases_nt[1]),
                len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2])/len(separacao_fases_nt[2])]

points_pb = [[], []]
points_pb[0] = [1 - (len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]))/( len(separacao_fases_pb[0]) + len(separacao_fases_pb[2]) ),
                1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]))/( len(separacao_fases_pb[0]) + len(separacao_fases_pb[1]) ),
                1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]))/( len(separacao_fases_pb[1]) + len(separacao_fases_pb[2]) )]
points_pb[1] = [len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0])/len(separacao_fases_pb[0]),
                len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1])/len(separacao_fases_pb[1]),
                len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2])/len(separacao_fases_pb[2])]

In [ ]:
# Plot

fig = plt.figure()
fig.set_size_inches(8, 8)
fig.suptitle("ROC - Seg. Length - Series 3")
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel("1 - Specificity"); ax1.set_ylabel("Sensitivity")
ax1.set_xlim(0,1); ax1.set_ylim(0, 1)

markers = ['o', 'x', 'D']
labels  = ["0", "1", ""]

ax1.plot(np.arange(0, 1.1, 0.1), np.arange(0, 1.1, 0.1), 'k--')
for i in range(3):
    ax1.scatter(points_ht[0][i], points_ht[1][i], marker = markers[i], color = 'k', label = labels[i]+" - Hypertensive")
    ax1.scatter(points_nt[0][i], points_nt[1][i], marker = markers[i], color = 'b', label = labels[i]+" - Normotensive")
    ax1.scatter(points_pb[0][i], points_pb[1][i], marker = markers[i], color = 'r', label = labels[i]+" - Proband")
    
ax1.legend()    
fig.savefig("Plots/Medidas/3 Grupos/roc_serie3_tamanho.png")

### Curva ROC 5 Fases - Eficiência de classificação via verificação direta do tamanho do segmento.

In [ ]:
media = []
tamanho = []
fase = []
todos = [[], [], []]
series = 3

# 1: pegar os valores pertinentes de todos os individuos, separados por grupos.
for j in range(3):
    for i in range(len(data[j])):
        media = media + list(seg_res[j][i][series]["mean"].values)
        tamanho = tamanho + list( seg_res[j][i][series]["finish"] - seg_res[j][i][series]["start"] )
        fase = fase + list(seg_res[j][i][series]["SleepStage"])
    todos[j] = pd.DataFrame(columns = ["media", "tamanho", "fase ref", "prev"])
    todos[j]["media"] = media; todos[j]["tamanho"] = np.log(np.asarray(tamanho)); todos[j]["fase ref"] = fase
    
# 2: adapta a fase 5 para fase 4    
for i in range(3):
    for j in range(len(todos[i])):
        if (todos[i]["fase ref"][j] == 5.0):
            todos[i].loc[j, "fase ref"] = 4.0

# 3: separo as tabelas obtidas anteriormente pelas fases.            
separacao_fases_ht = [[], [], [], [], []]
separacao_fases_nt = [[], [], [], [], []]
separacao_fases_pb = [[], [], [], [], []]
for i in range(5):
    separacao_fases_ht[i] = todos[0][(todos[0]["fase ref"] == i)]
    separacao_fases_nt[i] = todos[1][(todos[1]["fase ref"] == i)]
    separacao_fases_pb[i] = todos[2][(todos[2]["fase ref"] == i)]

In [ ]:
# serie 0
#ht_mean = np.array([3.85, 3.90, 4.14, 4.24, 3.82])
#nt_mean = np.array([3.83, 3.79, 4.18, 4.22, 3.80])
#pb_mean = np.array([3.73, 3.81, 3.95, 3.94, 3.80])

# serie 1
#ht_mean = np.array([3.91, 4.04, 4.32, 4.40, 3.84])
#nt_mean = np.array([3.93, 3.95, 4.32, 4.24, 3.88])
#pb_mean = np.array([3.82, 3.81, 4.05, 4.10, 3.90])

# serie 2
#ht_mean = np.array([3.77, 3.87, 4.29, 4.48, 3.76])
#nt_mean = np.array([3.84, 4.02, 4.72, 4.82, 3.97])
#pb_mean = np.array([3.89, 4.06, 4.45, 4.65, 3.89])

# serie 3
ht_mean = np.array([3.72, 3.87, 4.22, 4.40, 3.72])
nt_mean = np.array([3.84, 3.99, 4.64, 4.67, 3.94])
pb_mean = np.array([3.89, 4.06, 4.45, 4.65, 3.89])

In [ ]:
# 1: faço uma previsão por comparação das diferenças absolutas do tamanho com o tamanho médio.
@jit
def prev(ht_mean, nt_mean, pb_mean, separacao_fases_ht, separacao_fases_nt, separacao_fases_pb):
    for j in range(5):
        for i in separacao_fases_ht[j].index:
            separacao_fases_ht[j].loc[i,"prev"] = np.argmin(np.abs(ht_mean - separacao_fases_ht[j]["tamanho"].loc[i]))
        for i in separacao_fases_nt[j].index:
            separacao_fases_nt[j].loc[i,"prev"] = np.argmin(np.abs(nt_mean - separacao_fases_nt[j]["tamanho"].loc[i]))
        for i in separacao_fases_pb[j].index:
            separacao_fases_pb[j].loc[i,"prev"] = np.argmin(np.abs(pb_mean - separacao_fases_pb[j]["tamanho"].loc[i]))

In [ ]:
prev(ht_mean, nt_mean, pb_mean, separacao_fases_ht, separacao_fases_nt, separacao_fases_pb)

In [ ]:
# 2: calculos os pontos para a ROC, avaliando a eficiência da previsão para cada fase, para cada grupo        
points_ht = [[], []]
points_ht[0] = [1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]) + len(separacao_fases_ht[3][separacao_fases_ht[3]["prev"] == 3]) + len(separacao_fases_ht[4][separacao_fases_ht[4]["prev"] == 4]))/( len(separacao_fases_ht[1]) + len(separacao_fases_ht[2]) + len(separacao_fases_ht[3]) + len(separacao_fases_ht[4]) ),
                1 - (len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]) + len(separacao_fases_ht[3][separacao_fases_ht[3]["prev"] == 3]) + len(separacao_fases_ht[4][separacao_fases_ht[4]["prev"] == 4]))/( len(separacao_fases_ht[0]) + len(separacao_fases_ht[2]) + len(separacao_fases_ht[3]) + len(separacao_fases_ht[4]) ),
                1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]) + len(separacao_fases_ht[3][separacao_fases_ht[3]["prev"] == 3]) + len(separacao_fases_ht[4][separacao_fases_ht[4]["prev"] == 4]))/( len(separacao_fases_ht[1]) + len(separacao_fases_ht[0]) + len(separacao_fases_ht[3]) + len(separacao_fases_ht[4]) ),
                1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]) + len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]) + len(separacao_fases_ht[4][separacao_fases_ht[4]["prev"] == 4]))/( len(separacao_fases_ht[1]) + len(separacao_fases_ht[2]) + len(separacao_fases_ht[0]) + len(separacao_fases_ht[4]) ),
                1 - (len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1]) + len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2]) + len(separacao_fases_ht[3][separacao_fases_ht[3]["prev"] == 3]) + len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0]))/( len(separacao_fases_ht[1]) + len(separacao_fases_ht[2]) + len(separacao_fases_ht[3]) + len(separacao_fases_ht[0]) )]
points_ht[1] = [len(separacao_fases_ht[0][separacao_fases_ht[0]["prev"] == 0])/len(separacao_fases_ht[0]),
                len(separacao_fases_ht[1][separacao_fases_ht[1]["prev"] == 1])/len(separacao_fases_ht[1]),
                len(separacao_fases_ht[2][separacao_fases_ht[2]["prev"] == 2])/len(separacao_fases_ht[2]),
                len(separacao_fases_ht[3][separacao_fases_ht[3]["prev"] == 3])/len(separacao_fases_ht[3]),
                len(separacao_fases_ht[4][separacao_fases_ht[4]["prev"] == 4])/len(separacao_fases_ht[4])]

points_nt = [[], []]
points_nt[0] = [1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]) + len(separacao_fases_nt[3][separacao_fases_nt[3]["prev"] == 3]) + len(separacao_fases_nt[4][separacao_fases_nt[4]["prev"] == 4]))/( len(separacao_fases_nt[1]) + len(separacao_fases_nt[2]) + len(separacao_fases_nt[3]) + len(separacao_fases_nt[4]) ),
                1 - (len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]) + len(separacao_fases_nt[3][separacao_fases_nt[3]["prev"] == 3]) + len(separacao_fases_nt[4][separacao_fases_nt[4]["prev"] == 4]))/( len(separacao_fases_nt[0]) + len(separacao_fases_nt[2]) + len(separacao_fases_nt[3]) + len(separacao_fases_nt[4]) ),
                1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]) + len(separacao_fases_nt[3][separacao_fases_nt[3]["prev"] == 3]) + len(separacao_fases_nt[4][separacao_fases_nt[4]["prev"] == 4]))/( len(separacao_fases_nt[1]) + len(separacao_fases_nt[0]) + len(separacao_fases_nt[3]) + len(separacao_fases_nt[4]) ),
                1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]) + len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]) + len(separacao_fases_nt[4][separacao_fases_nt[4]["prev"] == 4]))/( len(separacao_fases_nt[1]) + len(separacao_fases_nt[2]) + len(separacao_fases_nt[0]) + len(separacao_fases_nt[4]) ),
                1 - (len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1]) + len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2]) + len(separacao_fases_nt[3][separacao_fases_nt[3]["prev"] == 3]) + len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0]))/( len(separacao_fases_nt[1]) + len(separacao_fases_nt[2]) + len(separacao_fases_nt[3]) + len(separacao_fases_nt[0]) )]
points_nt[1] = [len(separacao_fases_nt[0][separacao_fases_nt[0]["prev"] == 0])/len(separacao_fases_nt[0]),
                len(separacao_fases_nt[1][separacao_fases_nt[1]["prev"] == 1])/len(separacao_fases_nt[1]),
                len(separacao_fases_nt[2][separacao_fases_nt[2]["prev"] == 2])/len(separacao_fases_nt[2]),
                len(separacao_fases_nt[3][separacao_fases_nt[3]["prev"] == 3])/len(separacao_fases_nt[3]),
                len(separacao_fases_nt[4][separacao_fases_nt[4]["prev"] == 4])/len(separacao_fases_nt[4])]

points_pb = [[], []]
points_pb[0] = [1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]) + len(separacao_fases_pb[3][separacao_fases_pb[3]["prev"] == 3]) + len(separacao_fases_pb[4][separacao_fases_pb[4]["prev"] == 4]))/( len(separacao_fases_pb[1]) + len(separacao_fases_pb[2]) + len(separacao_fases_pb[3]) + len(separacao_fases_pb[4]) ),
                1 - (len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]) + len(separacao_fases_pb[3][separacao_fases_pb[3]["prev"] == 3]) + len(separacao_fases_pb[4][separacao_fases_pb[4]["prev"] == 4]))/( len(separacao_fases_pb[0]) + len(separacao_fases_pb[2]) + len(separacao_fases_pb[3]) + len(separacao_fases_pb[4]) ),
                1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]) + len(separacao_fases_pb[3][separacao_fases_pb[3]["prev"] == 3]) + len(separacao_fases_pb[4][separacao_fases_pb[4]["prev"] == 4]))/( len(separacao_fases_pb[1]) + len(separacao_fases_pb[0]) + len(separacao_fases_pb[3]) + len(separacao_fases_pb[4]) ),
                1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]) + len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]) + len(separacao_fases_pb[4][separacao_fases_pb[4]["prev"] == 4]))/( len(separacao_fases_pb[1]) + len(separacao_fases_pb[2]) + len(separacao_fases_pb[0]) + len(separacao_fases_pb[4]) ),
                1 - (len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1]) + len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2]) + len(separacao_fases_pb[3][separacao_fases_pb[3]["prev"] == 3]) + len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0]))/( len(separacao_fases_pb[1]) + len(separacao_fases_pb[2]) + len(separacao_fases_pb[3]) + len(separacao_fases_pb[0]) )]
points_pb[1] = [len(separacao_fases_pb[0][separacao_fases_pb[0]["prev"] == 0])/len(separacao_fases_pb[0]),
                len(separacao_fases_pb[1][separacao_fases_pb[1]["prev"] == 1])/len(separacao_fases_pb[1]),
                len(separacao_fases_pb[2][separacao_fases_pb[2]["prev"] == 2])/len(separacao_fases_pb[2]),
                len(separacao_fases_pb[3][separacao_fases_pb[3]["prev"] == 3])/len(separacao_fases_pb[3]),
                len(separacao_fases_pb[4][separacao_fases_pb[4]["prev"] == 4])/len(separacao_fases_pb[4])]

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
fig.set_size_inches(8, 8)
fig.suptitle("ROC - Seg. Length - Series 3")
ax1 = fig.add_subplot(1,1,1)
ax1.set_xlabel("1 - Specificity"); ax1.set_ylabel("Sensitivity")
ax1.set_xlim(0,1); ax1.set_ylim(0, 1)

markers = ['o', 'x', 'D', 's', 'P']
labels  = ['0', '1', '2', '3', 'REM']

ax1.plot(np.arange(0, 1.1, 0.1), np.arange(0, 1.1, 0.1), 'k--')
for i in range(5):
    ax1.scatter(points_ht[0][i], points_ht[1][i], marker = markers[i], color = 'k', label = labels[i]+" - Hypertensive")
    ax1.scatter(points_nt[0][i], points_nt[1][i], marker = markers[i], color = 'b', label = labels[i]+" - Normotensive")
    ax1.scatter(points_pb[0][i], points_pb[1][i], marker = markers[i], color = 'r', label = labels[i]+" - Proband")

ax1.legend()
fig.savefig("Plots/Medidas/5 Grupos/roc_serie3_tamanho.png")